In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from textblob import TextBlob
import multiprocessing as mp
import pickle
import sklearn.feature_extraction.text as text
from sklearn import decomposition
import seaborn as sns
import nltk
from nltk.tokenize import wordpunct_tokenize

# Pickleload

In [3]:
pickloader = open("pickles/amazon.pkl","rb")
amazon = pickle.load(pickloader)
pickloader.close()

# Picklesave



In [ ]:
# pickler = open("pickles/amazon.pkl","wb")
# pickle.dump(amazon, pickler)
# pickler.close()

In [8]:
reviews = amazon[['asin','reviewText','overall']]
books = set(reviews['asin'].unique())
book_list = list(books)

In [9]:
print(reviews.shape)
reviews = reviews.dropna()

(25712622, 3)


In [37]:
print(reviews.shape)
reviews.reviewText[2].lower()

(19526090, 3)


"worth the money the pros:1. it really teaches you the basics on cooking2. really easy to make3. the recipes are categorized which make it easy to findthe cons:1. some recipes are really high in calories2. don't seem all that cheap to me, speaking from a college student on a budget in less time and for less money than it takes to order pizza, you can make it yourself!three harried but heatlh-conscious college students compiled and tested this collection of more than 200 tasty, hearty, inexpensive recipes anyone can cook -- yes, anyone!whether you're short on cash, fearful of fat, counting your calories, or just miss home cooking, the healthy college cookbook offers everything you need to make good food yourself."

In [39]:
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "i would",
  "i'd've": "i would have",
  "i'll": "i will",
  "i'll've": "i will have",
  "i'm": "i am",
  "i've": "i have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

clean_reviews_expanded = [expandContractions(review.lower()) for review in reviews.reviewText]

clean_reviews = clean_reviews_expanded

In [44]:
len(clean_reviews)

19526090

# pickle save

In [42]:
pickler = open("pickles/clean_reviews.pkl","wb")
pickle.dump(clean_reviews, pickler)
pickler.close()

# pickle load

In [3]:
pickloader = open("pickles/clean_reviews.pkl","rb")
clean_reviews = pickle.load(pickloader)
pickloader.close()

In [4]:
# Lemming: a gentler approach: 
lemma=nltk.stem.WordNetLemmatizer()

lemma_books = []

# if __name__ == '__main__':
#     with mp.Pool() as pool:
#         tokens = pool.map(lemma.lemmatize(lemma.lemmatize(), clean_reviews))
for book in clean_reviews:
    lemma_book = [lemma.lemmatize(word) for word in wordpunct_tokenize(book)]
    lemma_book = (' ').join(lemma_book)
    lemma_books.append(lemma_book)

In [5]:
# pickler = open("pickles/lemma_books.pkl","wb")
# pickle.dump(lemma_books, pickler)
# pickler.close()

In [ ]:
pickloader = open("pickles/lemma_books.pkl","rb")
lemma_books = pickle.load(pickloader)
pickloader.close()